Importation des bibliothèques nécessaires, et calcul des caractéristiques du modèle. Attention : NE PAS EFFACER !

In [ ]:
from pylab import *    # charge toutes les bibliothèques scientifiques
from scipy.optimize import curve_fit    # permet de modéliser selon un modèle personnalisé
from lmfit import minimize,Parameters,Parameter,report_fit
from lmfit.models import ExpressionModel
import re

# Si on veut différents graphiques non interactifs dans la feuille, remplacer ipympl par inline dans la ligne suivante
%matplotlib ipympl
rcParams['figure.figsize'] = [16, 8]    # pour une fenetre graphique adaptée
rcParams['font.size'] = 15    # pour un texte plus grand
rcParams['lines.markersize'] = 15    # pour des symboles plus grands
rcParams['lines.markeredgewidth'] = 2    # pour des symboles plus larges

# Calculs du modèle. Attention : NE JAMAIS MODIFIER LES LIGNES SUIVANTES !
def Modele(expression, x, y, contraintes):
    
    modele = ExpressionModel(expression)    # si la variable indépendante n'est pas "x", ajouter comme argument independant_vars = ["nom"])

    # Cette partie devine les paramètres et leur attribue une valeur initiale de 1 pour la recherche
    parametres = modele.make_params()
    for i in parametres :
        modele.set_param_hint(i, value = 1)

    for j in contraintes :
        if j[0] in parametres :
            modele.set_param_hint(j[0], value = j[1], vary = j[2], min = j[3], max = j[4])    # À compléter pour changer les valeurs initiales des paramètres. Le dernier argument est expr (string)

    parametres = modele.make_params()
    
    resultat = modele.fit(y, parametres, x = x)    # Entrer ici le nom de la variable en x et en y, pour le calcul final des paramètres

    # On affiche les valeurs trouvées pour les paramètres :
    valeurs = ""
    for key in resultat.params:
        valeurs += f"{key} = {resultat.params[key].value:.3g} ; incertitude : {resultat.params[key].stderr:.2g}\n"
    return(modele, resultat.params, valeurs, expression)

def Calcul_modele(abscisse,  ordonnee, equation, debut, fin, debutCourbe, finCourbe, contraintes):
    ## On commence par arranger les expressions avec "x" comme variable, car lmfit travaille avec x par défaut :
    ord = ordonnee
    eq = equation
    equation = re.sub(r"\b"+abscisse+r"\b","x", equation)
    abscisse = globals()[abscisse]
    ordonnee = globals()[ordonnee]
    if debutCourbe == None :
        debutCourbe = min(abscisse)
    if finCourbe == None :
        finCourbe = max(abscisse)
    
    # On calcule maintenant les grandeurs liées au modèle :
    xMod = linspace(debutCourbe, finCourbe, 30)    # Intervalle de valeurs pour le modèle : (début, fin, nombre de points)
    modele, parametres, valeurs, expression = Modele(equation, abscisse[debut:fin], ordonnee[debut:fin], contraintes)    # Expression du modèle, et variables en abscisse et en ordonnée ; on peut changer les plages de valeurs à utiliser pour le calcul
    expression = f"{ord} = {eq}"
    yMod = modele.eval(parametres, x = xMod)
    
    # Les paramètres sont définis de façon globale, et utilisables dans des calculs :
    for key in parametres:
        globals()[key] = parametres[key].value
    
    return(xMod, yMod, expression, valeurs, abscisse, ordonnee, modele, parametres)

Importation de tableaux enregistrés localement en fichiers csv :

In [ ]:
noms = True    # si le fichier csv contient les noms des variables, laisser True, sinon indiquer False
tableau = genfromtxt('untitled.csv', delimiter=',', skip_header=0, names=noms)    # skip_header permet de ne pas lire les n premières lignes du fichier (utile si il y a des en-têtes) ; delimiter est le signe séparant les valeurs dans les différentes colonnes : par exemple, on peut parfois mettre delimiter=None si les séparateurs sont des espaces
    
if noms :    # dans le cas d'attribution automatique des noms
    for i in tableau.dtype.names:    # pour chaque nom détecté en en-tete de colonnes
        globals()[i] = tableau[i]    # on crée une variable qui porte le meme nom et on y attribue les valeurs de la colonne

else :    # si on veut attribuer les noms manuellement
    x = tableau[:, 0] # On enregistre la première colonne sous le nom x
    y = tableau[:, 1] # On enregistre la deuxième colonne sous le nom y    # etc... s'il y a d'autres colonnes

Tracé de graphique(s) à partir des tableaux précédents :

In [ ]:
abscisse = "x"    # choix de la grandeur en abscisse, pour le graphique et pour le modèle
ordonnee = "y"    # choix de la grandeur en ordonnée, pour le graphique et pour le modèle
equation = "a * x + b"    # expression de l'équation correspondant au modèle recherché ; la variable peut être x ou le nom donné en abscisse
debut = None    # indice du premier élément à prendre en compte pour le modèle (None : le premier de la liste)
fin = None    # indice du dernier élément à prendre en compte pour le modèle (None : le dernier de la liste)
debutCourbe = None    # valeur de l'abscisse où débute le tracé du modèle (None : la plus petite valeur de l'abscisse)
finCourbe = None    # valeur de l'abscisse où se termine le tracé du modèle (None : la plus grande valeur de l'abscisse)
contraintes = [['a', 1, True, -inf, inf]]    # ['nomDuParametre', valeurInitiale, variabilité, min, max]   à remplir pour chaque paramètre du modèle si la recherche automatique ne trouve pas de valeurs cohérentes

subplot(1,1,1)    # création d'une fenêtre de graphique : lignes, colonnes, numéro
xMod, yMod, expression, valeurs, abscisse, ordonnee, modele, parametres = Calcul_modele(abscisse, ordonnee, equation, debut, fin, debutCourbe, finCourbe, contraintes)    # Calcul des valeurs du modèle, ne pas modifier

# Tracé des points expérimentaux :
plot(abscisse, ordonnee, '+', label="")    # abscisse, ordonnée, options : + o . , s points   - -- -. : lignes  b g r y m c k w couleurs ; label est la légende associée à la courbe

# Tracé du modèle :
plot(xMod, yMod, '-', label=f"Modèle :    {expression} \n {valeurs}" )    # trace la courbe du modèle

#axis('equal')    # à décommenter si on veut un repère orthonormé
grid()    # ajoute une grille
legend()    # ajoute la légende au graphique
title("Mon titre")    # mettre le titre désiré entre les guillemets
xlabel("")    # mettre le nom de l'axe des abscisses entre les guillemets
ylabel("");    # mettre le nom de l'axe des ordonnées entre les guillemets. Le point-virgule final évite d'afficher un texte récapitulatif.

Calcul de valeurs particulières :

In [ ]:
# Calcul dans le sens direct :
valeur_de_x = 1    # attribuer un nombre ou une liste de nombres à valeur_de_x 
valeur_de_y = modele.eval(parametres, x = valeur_de_x)    
print(valeur_de_y)